## An example workflow

This is a basic workflow for runing a D3D simulation. Since no meteo or bathymetry source is defined the data are downloaded by an erddap server.

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.model as pm
from pyPoseidon.utils import data,point

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [ ]:
from hvplot import xarray, pandas

In [ ]:
pyPoseidon.__version__

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-29., # lat/lon window
     'maxlon':-9.,
     'minlat':56.,
     'maxlat':74.,
     'start_date':'2018-10-1',
     'time_frame':'12H',
#     'end_date':'2018-10-5' #optional end_date instead of time_frame
     'solver':'d3d',
     'resolution':0.2, #grid resoltuion 
     'step':20, # step for output of map field in d3d     
     'ncores': 4 , #number of cores
     'rpath':'./test/d3d/', #location of calc folder
     'epath':'/Users/brey/DELFT3D/SVN/7545/bin/lnx64/', #folder for solver executables
     'conda_bin':'/Users/brey/miniconda3/bin/',
     'conda_env':'mpi3' # optional conda env for running the solver
    }

### Create model

In [ ]:
#initialize a model
b = pm.model(**dic)

### set it up

In [ ]:
b.set() #set it up 

In [ ]:
b.impl.meteo.uvp

### output to rpath 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

### Visualise

In [ ]:
otp = data(**dic)

In [ ]:
otp.impl.Dataset

#### Grid & Bathymetry 

In [ ]:
otp.impl.dem.bathymetry.hvplot.contourf('longitude','latitude',cmap='viridis', height=600)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

otp.impl.dem.bathymetry.plot.pcolormesh('longitude', 'latitude', ax=ax);

gp = ax.scatter(otp.impl.grid.lons, otp.impl.grid.lats, s=2, color='k',  label='grid points', transform=ccrs.PlateCarree());

#pp = ax.scatter(otp.impl.xh, otp.impl.yh, s=2, color='b', label='pressure points', transform=ccrs.PlateCarree());

ax.legend()

ax.coastlines('50m'); ax.gridlines(draw_labels=True);

#### Variable snapshot

#### Holoviews

In [ ]:
otp.impl.Dataset.S1.sel(M=slice(1,-1)).sel(N=slice(1,-1)).hvplot.contourf(x='XZ',y='YZ',cmap='viridis')#.hist()

#### Geoviews

In [ ]:
#with map and zooming but dynamic colorbar to make it fast, otherwise slow and heavy
g50 = gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
otp.impl.Dataset.S1[:,1:-1,1:-1].hvplot.quadmesh(x='XZ',y='YZ',cmap='viridis', geo=True) * g50 * gf.borders()

In [ ]:
otp.impl.Dataset['S1'] = otp.impl.Dataset.S1.transpose('time','N','M')

#### Inherent Animation option

In [ ]:
# relatively fast and you can save in mp4
otp.impl.frames(['S1'],title='SSH')
#otp.impl.frames(['S1'],title='SSH',savepath='./test/test.mp4')

In [ ]:
# relatively fast and you can save in mp4
otp.impl.frames(['U1','V1'],title='Sea Vectors',scale=.1)

### Observation points

In [ ]:
otp.impl.obs.locations # in-situ measurements available within the lat/lon window

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(1)
p.head()

In [ ]:
#... or get with Station Name  
p = otp.impl.obs.loc('Iceland - Reykjavik')
p.head()

In [ ]:
#plot
ax = p.plot()

#### sample nearest point from simulation 

In [ ]:
lat,lon = otp.impl.obs.locations.loc[1,['lat','lon']] #get lat lon from tide gauge
print (lat, lon)

In [ ]:
# Fast way to interpolate within xarray. It could produce NaNs though...
s1 = otp.impl.Dataset.S1 # select variable
s1 = s1.sel(M=slice(1,-1)).sel(N=slice(1,-1)) # drop first & last element (zeros)
s1 = s1.assign_coords(M=s1.XZ[:,0].values, N=s1.YZ[0,:].values) # change coords to lat/lon from index
s1 = s1.drop(['XZ','YZ']).rename({'M':'longitude','N':'latitude'}) # rename dims


near = s1.sel(longitude=[lon], method='nearest').sel(latitude=[lat], method='nearest')

#Check ts, could be nan
near = near.squeeze().to_pandas()
near.head()

Other option with point class

In [ ]:
ts = point(lon=lon,lat=lat,data=otp.impl) #This class uses pyresample and works for all locations.
ts.tseries(var='S1')
ts.S1.head()

In [ ]:
ax = ts.S1.plot()
near.plot(ax=ax)

In [ ]:
## Join the graphs
ax = ts.S1.plot(figsize=(10,8),color=['r'])
p.plot(ax=ax)
ax.legend(['D3D'] + list(p.columns.values), loc='center left',bbox_to_anchor=(1.1, 0.5))

.. or ..

In [ ]:
## Join the graphs
p.hvplot(figsize=(14,10)) * ts.S1.hvplot(label='d3d',color='red')